In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from keras import regularizers
from ImageDataAugmentor.image_data_augmentor import *
import tensorflow_datasets as tfds
from functools import partial
import albumentations as A
import cv2
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def append_ext(fn):
    return fn+".jpg"



In [ ]:
i2019=pd.read_csv("../input/isic-2019-challenge/ISIC_2019_Training_GroundTruth.csv")
train=pd.read_csv("../input/isic2019-manip/isic2019_train_onlyisic")
valid=pd.read_csv('../input/isic2019-manip/isic2019_valid_onlyisic')
dir_imgs  = "../input/isic-2019-challenge/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

In [ ]:
train["image_id"]=train["image_id"].apply(append_ext)
valid["image_id"]=valid["image_id"].apply(append_ext)

In [ ]:
batch_size=32
img_res=224
img_format=(img_res,img_res)
checkpoint_dir=''
save_dir=''

In [ ]:
net= tf.keras.applications.MobileNetV3Small(weights='imagenet',include_top=False, input_shape=(img_res,img_res,3))
model=Model(inputs=net.input,outputs=net.output )
for layers in model.layers:
    layers.trainable=False
    
x=net.output
x = GlobalAveragePooling2D()(x)
#x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
#x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)
model = Model(inputs=net.input, outputs=predictions)

In [ ]:
preprocess = tf.keras.applications.mobilenet_v3.preprocess_input

In [ ]:
    image_size=img_res
    augmentation = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.3, p=0.75),
        albumentations.RandomContrast(limit=0.3, p=0.75),
       albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=3),
            albumentations.MedianBlur(blur_limit=3),
#            albumentations.GaussianBlur(blur_limit=3),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
       ], p=0.7),

#        albumentations.OneOf([
#            albumentations.OpticalDistortion(distort_limit=1.0),
#            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#            albumentations.ElasticTransform(alpha=3),
#        ], p=0.7),

        #albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
    ])



In [ ]:
train_datagen=ImageDataAugmentor(
                           augment=augmentation,
                           preprocess_input=preprocess
                                )



train_generator=train_datagen.flow_from_dataframe(
dataframe=train,
directory=dir_imgs,
x_col="image_id",
y_col="dx",
batch_size=batch_size,
#seed=42,
shuffle=True,
class_mode="categorical",
target_size=(img_res,img_res)
)

In [ ]:
valid_datagen=ImageDataGenerator(
    preprocessing_function=preprocess)



validation_generator = valid_datagen.flow_from_dataframe(
dataframe=valid,
directory=dir_imgs,
x_col="image_id",
y_col="dx",
batch_size=batch_size,
shuffle=False,
class_mode="categorical",
target_size=(img_res,img_res)
)

In [ ]:

checkpoint_dir= './checkpoint/'

model.compile(loss='categorical_crossentropy',
  optimizer=Adam(learning_rate=0.001),
  metrics=['accuracy','AUC',top_2_accuracy, top_3_accuracy])



checkpoint = ModelCheckpoint(checkpoint_dir, 
                        monitor='val_accuracy', verbose=1, 
                        save_best_only=True, mode='max', save_freq='epoch')

callbacks_list = [checkpoint]

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
history=model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2,
                    callbacks=callbacks_list

)

In [ ]:
for layers in model.layers:
    layers.trainable=True

In [ ]:

checkpoint_dir= './checkpoint2/'

model.compile(loss='categorical_crossentropy',
  optimizer=Adam(learning_rate=0.0001),
  metrics=['accuracy','AUC',top_2_accuracy, top_3_accuracy, tf.keras.metrics.Precision(),  tf.keras.metrics.Recall()])

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10)

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,
                          patience=2, min_lr=0.0000001)

checkpoint = ModelCheckpoint('checkpoint2.h5', 
                        monitor='val_accuracy', verbose=1, 
                        save_best_only=True, mode='max', save_freq='epoch')

callbacks_list = [checkpoint, earlyStopping, reduce_lr]
    

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
history=model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
#                    class_weight=class_weights,
                    epochs=40,
                    callbacks=callbacks_list

)

In [ ]:
model.save('./mobilenetv3.h5')